Στόχος είναι να χαρακτηρίσουμε με "έξυπνο" τρόπο το σύνολο των κειμένων με τις οντότητες που περιέχει το καθένα (και τις θέσεις αυτών στο κείμενο) ώστε να μπορέσει να χρησιμοποιηθεί σε επόμενο στάδιο για την εκπαίδευση του Named Entity Recognizer. Αυτή η επισήμανση μπορεί να γίνει με αυτόματο τρόπο, πραγματοποιώντας αναζήτηση στα κείμενα με βάση γνωστές οντότητες που μας ενδιαφερούν στην παρούσα εργασία (βότανα, όργανα ανθρώπινου σώματος κλπ.).

Πρώτα φτιάχνουμε ένα csv file (organ_systems.csv) με όλα τα συστήματα οργάνων του ανθρώπινου οργανισμού, τα όργανα που περιλαμβάνει το καθένα, τις αντίστοιχιες λειτουργίες και κάποια παράγωγα τους.

In [1]:
import pandas as pd

df_organs = pd.read_csv('../data/organ_systems.csv', encoding='ISO8859-7')
df_organs

,ΣΥΣΤΗΜΑΤΑ ΟΡΓΑΝΩΝ,ΟΡΓΑΝΑ,ΟΡΓΑΝΙΚΕΣ ΛΕΙΤΟΥΡΓΙΕΣ,ΟΡΓΑΝΙΚΑ ΠΑΡΑΓΩΓΑ
0,αισθητήριο,"μάτια,αυτιά,γλώσσα,μύτη","όραση,ακοή,όσφρηση",NaN
1,νευρικό,"νεύρα,εγκέφαλος,μυελός,μυαλό","σκέψη,μνήμη,αντίληψη",NaN
2,ενδοκρινών,"ενδοκρινείς,υποθάλαμος,υπόφυση,θυρεοειδής,παρα...",NaN,ορμόνες
3,πεπτικό,"στομάχι,σιελογόνοι,οισοφάγος,συκώτι,πάγκρεας,ή...","χώνεψη,πέψη",NaN
4,κυκλοφορικό,"καρδιά,αιμοφόρα,αρτηρίες,φλέβες,τριχοειδή,αγγεία",κυκλοφορία,αίμα
5,λεμφικό,"αμυγδαλές,σπλήνα,θύμος",άμυνα,αντισώματα
6,αναπνευστικό,"ρινική,φάρυγγας,λάρυγγας,τραχεία,βρόγχος,πνεύμ...",αναπνοή,οξυγόνο
7,ουροποιητικό,"νεφρά,ουρητήρες,ουροδόχος,ουρήθρα",ούρηση,ούρα
8,ερειστικό,"οστά,σύνδεσμοι,τένοντες,χόνδρος,κόκαλα,σκελετό...",NaN,NaN
9,μυϊκό,"μύες,μυς",κίνηση,NaN


Σε πρώτη φάση θα αναζητήσουμε μόνο τα όργανα που εμφανίζονται μέσα στα κείμενα από τη σελίδα "Προϊόντα της Φύσης".

In [41]:
df_texts = pd.read_csv('../data/proionta_tis_fisis_votana.csv')

Θα εξάγουμε μια λίστα με τα όργανα-όρους αναζήτησης και θα χρειαστεί διασπάσουμε τα κείμενα σε λέξεις (tokens). Αυτό γίνεται με τον tokenizer της spaCy για τα Ελληνικά. Επειδή όμως οι λέξεις μπορεί να βρίσκονται σε διάφορες πτώσεις πρέπει να κάνουμε lemmatization ή stemming τόσο στους όρους αναζήτησης όσο και στις λέξεις των κειμένων, ώστε να είναι επιτυχημένο το string matching που θα ακολουθήσει. 

Δοκιμάσαμε το lemmatizer της spaCy τόσο από το small όσο και από το medium model που διατίθενται για την ελληνική γλώσσα. Αν και το medium απέδωσε καλύτερα, γενικά η λημματοποίηση δεν είχε όσο ικανοποιητικά αποτελέσματα θα θέλαμε. Στα αποτελέσματα παρέμεναν διαφορετικές πτώσεις και πολλές φορές λανθασμένες παραποιήσεις της αρχικής λέξης, όπως θα δούμε στη συνέχεια. Για αυτό στραφήκαμε στη βιβλιοθήκη greek_stemmer που κάνει αποκατάληξη στα Ελληνικά, όπως λέει και το όνομά της. Συγκρίναμε τις δύο μεθόδους σε διάφορα παραδείγματα και όπως φαίνεται παρακάτω ο greek-stemmer αποδίδει καλύτερα και τα αποτελέσματά του φαίνονται να οδηγούν σε καλύτερο string matching αργότερα.

Και στις δύο μεθόδους παρατηρήθηκε ότι οι τόνοι μπορεί να οδηγούσαν σε διαφορετικά αποτελέσματα για την ίδια ρίζα, οπότε αφαιρέθηκαν προτού προχωρήσουμε με τη διαδικασία. Επίσης, η εναλλαγή πεζών κεφαλαίων κάποιες φορές οδήγησε σε διαφορετικά αποτελέσματα οπότε επιλέχθηκε να μετατραπούν όλες οι λέξεις σε μία από τις δύο μορφές. Μάλιστα, ο greek-stemmer φαίνεται να λειτουργεί μόνο με κεφαλαία οπότε επιλέχθηκε αυτή η μετατροπή.

Οι μικρές διαφορές στις πτώσεις και στις καταλήξεις θα μπορούσαν να αντιμετωπιστούν εφαρμόζοντας fuzzy string matching, με υψηλό κατώφλι χαλάρωσης. Ωστόσο υπάρχουν περιπτώσεις όπου απαιτούνταν χαμηλότερο κατώφλι για να ταιριάξουν δυο λέξεις με κοινή ρίζα ενώ ένα πολύ ψηλό επαρκούσε για να ταιριάξουν δύο τελειώς διαφορετικές λέξεις, όπως φαίνεται και στο παρακάτω παράδειγμα. Τέτοια προβλήματα εξακολούθησαν να υπάρχουν, πιο σπάνια βέβαια, ακόμα και όταν συνδυάστηκε η λημματοποίηση του spaCy με fuzzy string matching.

In [3]:
from fuzzywuzzy import process, fuzz
print(fuzz.ratio('έντερα', 'εντέρου'))
print(fuzz.ratio('καρδιά', 'καρυδιά'))

46
92


In [4]:
import spacy
import el_core_news_md
nlp = el_core_news_md.load()

In [5]:
from greek_stemmer import GreekStemmer
stemmer = GreekStemmer()

Παρακάτω παραθέτουμε κάποια παραδείγματα όπου φαίνεται η σύγκριση της λημματοποίησης με spaCy και της αποκατάληξης με greek_stemmer.

In [6]:
t = nlp('καρδια ΚΑΡΔΙΑ Καρδια καρδιας Καρδιας ΚΑΡΔΙΑΣ καρδιες ΚΑΡΔΙΕΣ καρδιων Καρδιων ΚΑΡΔΙΩΝ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΚΑΡΔΙΑ καρδιο ΚΑΡΔ
ΚΑΡΔΙΑ καρδιο ΚΑΡΔ
ΚΑΡΔΙΑ καρδιο ΚΑΡΔ
ΚΑΡΔΙΑΣ καρδια ΚΑΡΔ
ΚΑΡΔΙΑΣ ΚΑΡΔΙΑΣ ΚΑΡΔ
ΚΑΡΔΙΑΣ ΚΑΡΔΙΑΣ ΚΑΡΔ
ΚΑΡΔΙΕΣ ΚΑΡΔΙΕΣ ΚΑΡΔ
ΚΑΡΔΙΕΣ ΚΑΡΔΙΕΣ ΚΑΡΔ
ΚΑΡΔΙΩΝ ΚΑΡΔΙΩΝ ΚΑΡΔ
ΚΑΡΔΙΩΝ καρδιων ΚΑΡΔ
ΚΑΡΔΙΩΝ καρδιων ΚΑΡΔ


In [7]:
t = nlp('εντερα Εντερα εντερων ΕΝΤΕΡΑ Εντερων εντερων ΕΝΤΕΡΩΝ εντερο Εντερο ΕΝΤΕΡΟ εντερου Εντερου ΕΝΤΕΡΟΥ εντερου'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΕΝΤΕΡΑ εντερα ΕΝΤΕΡ
ΕΝΤΕΡΑ εντερα ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΑ εντερα ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΩΝ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ
ΕΝΤΕΡΟΥ εντερο ΕΝΤΕΡ


In [8]:
t = nlp('ουροποιητικο ουροποιητικου ουροποιητικου ουροποιητικου ΟΥΡΟΠΟΙΗΤΙΚΟΥ Ουροποιητικο'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΟΥΡΟΠΟΙΗΤΙΚΟ ουροποιητικος ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟΥ ουροποιητικο ΟΥΡΟΠΟΙΗΤ
ΟΥΡΟΠΟΙΗΤΙΚΟ ουροποιητικος ΟΥΡΟΠΟΙΗΤ


In [9]:
t = nlp('ενδοκρινεις Ενδοκρινεις Ενδοκρινων ΕΝΔΟΚΡΙΝΩΝ ενδοκρινων ενδοκρινης Ενδοκρινης ΕΝΔΟΚΡΙΝΗΣ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΕΝΔΟΚΡΙΝΕΙΣ ενδοκρινεις ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΕΙΣ ενδοκρινεις ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΩΝ ενδοκρινος ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΩΝ ενδοκρινος ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΩΝ ενδοκρινος ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝ
ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝΗΣ ΕΝΔΟΚΡΙΝ


In [10]:
t = nlp('ωοθηκες Ωοθηκες ΩΟΘΗΚΕΣ ωοθηκων ωοθηκη Ωοθηκων ΩΟΘΗΚΩΝ Ωοθηκη ωοθηκες ωοθηκες ωοθηκες ωοθηκες ωοθηκες'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΩΝ ωοθηκων ΩΟΘΗΚ
ΩΟΘΗΚΗ ωοθηκη ΩΟΘΗΚ
ΩΟΘΗΚΩΝ ωοθηκων ΩΟΘΗΚ
ΩΟΘΗΚΩΝ ωοθηκων ΩΟΘΗΚ
ΩΟΘΗΚΗ ωοθηκη ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ
ΩΟΘΗΚΕΣ ωοθηκε ΩΟΘΗΚ


In [11]:
t = nlp('ρινικος ρινικη Ρινικη ΡΙΝΙΚΗ ρινικες ΡΙΝΙΚΕΣ ρινικες ρινικες ρινικες'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΡΙΝΙΚΟΣ ρινικος ΡΙΝ
ΡΙΝΙΚΗ ρινικη ΡΙΝ
ΡΙΝΙΚΗ ρινικη ΡΙΝ
ΡΙΝΙΚΗ ρινικη ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ
ΡΙΝΙΚΕΣ ΡΙΝΙΚΕΣ ΡΙΝ


In [12]:
t = nlp('σμηγματογονος σμηγματογονοι Σμηγματαγονοι σμηγματογονων ΣΜΗΓΜΑΤΟΓΟΝΩΝ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΣΜΗΓΜΑΤΟΓΟΝΟΣ ΣΜΗΓΜΑΤΟΓΟΝΟΣ ΣΜΗΓΜΑΤΟΓΟΝ
ΣΜΗΓΜΑΤΟΓΟΝΟΙ ΣΜΗΓΜΑΤΟΓΟΝΟΙ ΣΜΗΓΜΑΤΟΓΟΝ
ΣΜΗΓΜΑΤΑΓΟΝΟΙ σμηγματαγονοι ΣΜΗΓΜΑΤΑΓΟΝ
ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝ
ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝΩΝ ΣΜΗΓΜΑΤΟΓΟΝ


In [13]:
t = nlp('αγκιναρα αγκιναρες ΑΓΚΙΝΑΡΕΣ αγκιναρων Αγκιναρας'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΑΓΚΙΝΑΡΑ αγκιναρο ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΕΣ αγκιναρε ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΕΣ αγκιναρε ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΩΝ αγκιναρων ΑΓΚΙΝΑΡ
ΑΓΚΙΝΑΡΑΣ αγκιναρας ΑΓΚΙΝΑΡ


In [14]:
t = nlp('Ανηθος ανηθου ανηθο ΑΝΗΘΟ'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΑΝΗΘΟΣ ανηθος ΑΝΗΘ
ΑΝΗΘΟΥ ανηθο ΑΝΗΘ
ΑΝΗΘΟ ανηθο ΑΝΗΘ
ΑΝΗΘΟ ανηθο ΑΝΗΘ


In [15]:
t = nlp('Δαμασκηνια δαμασκηνα ΔΑΜΑΣΚΗΝΟ δαμασκηνων Δαμασκηνου δμασκηνιας'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΔΑΜΑΣΚΗΝΙΑ δαμασκηνια ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΑ δαμασκηνα ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΟ δαμασκηνο ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΩΝ δαμασκηνο ΔΑΜΑΣΚΗΝ
ΔΑΜΑΣΚΗΝΟΥ δαμασκηνου ΔΑΜΑΣΚΗΝ
ΔΜΑΣΚΗΝΙΑΣ δμασκηνιας ΔΜΑΣΚΗΝ


In [16]:
t = nlp('κρεμμυδι Κρεμμυδα κρεμμυδια κρεμμυδιων Κρεμμυδιου'.upper())
for token in t:
    print(token.text,token.lemma_,stemmer.stem(token.text))

ΚΡΕΜΜΥΔΙ ΚΡΕΜΜΥΔΙ ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΑ κρεμμυδο ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΙΑ κρεμμυδια ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΙΩΝ κρεμμυδιων ΚΡΕΜΜΥΔ
ΚΡΕΜΜΥΔΙΟΥ κρεμμυδιου ΚΡΕΜΜΥΔ


Κατασκευάζουμε τη look up λίστα για τα όργανα, η οποία θα περιλαμβάνει όργανα-όρους αναζήτησης έτοιμους να χρησιμοποιηθούν για αναζήτηση, δηλαδή έπειτα από την όποια προεπεξεργασία αναφέραμε παραπάνω.

In [17]:
look_up_list1 = []
for i in range(df_organs.shape[0]):
    look_up_list1 += df_organs.iloc[i]['ΟΡΓΑΝΑ'].split(',')

In [18]:
look_up_list1

['μάτια',
 'αυτιά',
 'γλώσσα',
 'μύτη',
 'νεύρα',
 'εγκέφαλος',
 'μυελός',
 'μυαλό',
 'ενδοκρινείς',
 'υποθάλαμος',
 'υπόφυση',
 'θυρεοειδής',
 'παραθυροειδής',
 'επινεφρίδια',
 'επίφυση',
 'πάγκρεας',
 'θύμος',
 'βλεννογόνος',
 'ωοθήκες',
 'στομάχι',
 'σιελογόνοι',
 'οισοφάγος',
 'συκώτι',
 'πάγκρεας',
 'ήπαρ',
 'χοληδόχος',
 'έντερο',
 'καρδιά',
 'αιμοφόρα',
 'αρτηρίες',
 'φλέβες',
 'τριχοειδή',
 'αγγεία',
 'αμυγδαλές',
 'σπλήνα',
 'θύμος',
 'ρινική',
 'φάρυγγας',
 'λάρυγγας',
 'τραχεία',
 'βρόγχος',
 'πνεύμονας',
 'διάφραγμα',
 'νεφρά',
 'ουρητήρες',
 'ουροδόχος',
 'ουρήθρα',
 'οστά',
 'σύνδεσμοι',
 'τένοντες',
 'χόνδρος',
 'κόκαλα',
 'σκελετός',
 'αρθρώσεις',
 'δόντια',
 'μύες',
 'μυς',
 'δέρμα',
 'μαλλιά',
 'νύχια',
 'εξωκρινείς',
 'σμηγματογόνοι',
 'μήτρα',
 'μαστοί',
 'ωοθήκες',
 'όρχεις',
 'επιδιδυμίδα',
 'σάλπιγγες',
 'πέος',
 'προστάτης',
 'τράχηλος']

In [19]:
organs_look_up_list = []
for search_term in look_up_list1:
    search_term = search_term.replace('ά', 'α')
    search_term = search_term.replace('έ', 'ε')
    search_term = search_term.replace('ή', 'η')
    search_term = search_term.replace('ί', 'ι')
    search_term = search_term.replace('ό', 'ο')
    search_term = search_term.replace('ύ', 'υ')
    search_term = search_term.replace('ώ', 'ω')
    search_term = search_term.replace('ϊ', 'ι')
    search_term = search_term.replace('ΐ', 'ι')
    search_term = search_term.replace('ϋ', 'υ')
    search_term = search_term.replace('ΰ', 'υ')
    search_term = search_term.upper()
    organs_look_up_list += [stemmer.stem(search_term)]

In [20]:
organs_look_up_list

['ΜΑΤ',
 'ΑΥΤ',
 'ΓΛΩΣΣ',
 'ΜΥΤ',
 'ΝΕΥΡ',
 'ΕΓΚΕΦΑΛ',
 'ΜΥΕΛ',
 'ΜΥΑΛ',
 'ΕΝΔΟΚΡΙΝ',
 'ΥΠΟΘΑΛΑΜ',
 'ΥΠΟΦΥΣ',
 'ΘΥΡΕΟΕΙΔ',
 'ΠΑΡΑΘΥΡΟΕΙΔ',
 'ΕΠΙΝΕΦΡΙΔ',
 'ΕΠΙΦΥΣ',
 'ΠΑΓΚΡΕ',
 'ΘΥΜ',
 'ΒΛΕΝΝΟΓΟΝ',
 'ΩΟΘΗΚ',
 'ΣΤΟΜΑΧ',
 'ΣΙΕΛΟΓΟΝ',
 'ΟΙΣΟΦΑΓ',
 'ΣΥΚΩΤ',
 'ΠΑΓΚΡΕ',
 'ΗΠΑΡ',
 'ΧΟΛΗΔΟΧ',
 'ΕΝΤΕΡ',
 'ΚΑΡΔ',
 'ΑΙΜΟΦΟΡ',
 'ΑΡΤΗΡ',
 'ΦΛΕΒ',
 'ΤΡΙΧΟΕΙΔ',
 'ΑΓΓΕΙ',
 'ΑΜΥΓΔΑΛ',
 'ΣΠΛΗΝ',
 'ΘΥΜ',
 'ΡΙΝ',
 'ΦΑΡΥΓΓ',
 'ΛΑΡΥΓΓ',
 'ΤΡΑΧΕΙ',
 'ΒΡΟΓΧ',
 'ΠΝΕΥΜΟΝ',
 'ΔΙΑΦΡΑΓΜ',
 'ΝΕΦΡ',
 'ΟΥΡΗΤΗΡ',
 'ΟΥΡΟΔΟΧ',
 'ΟΥΡΗΘΡ',
 'ΟΣΤ',
 'ΣΥΝΔΕΣΜ',
 'ΤΕΝΟΝΤ',
 'ΧΟΝΔΡ',
 'ΚΟΚΑΛ',
 'ΣΚΕΛΕΤ',
 'ΑΡΘΡΩΣ',
 'ΔΟΝΤ',
 'ΜΥ',
 'Μ',
 'ΔΕΡΜ',
 'ΜΑΛΛ',
 'ΝΥΧ',
 'ΕΞΩΚΡΙΝ',
 'ΣΜΗΓΜΑΤΟΓΟΝ',
 'ΜΗΤΡ',
 'ΜΑΣΤ',
 'ΩΟΘΗΚ',
 'ΟΡΧ',
 'ΕΠΙΔΙΔΥΜΙΔ',
 'ΣΑΛΠΙΓΓ',
 'ΠΕ',
 'ΠΡΟΣΤΑΤ',
 'ΤΡΑΧΗΛ']

Την αντίστοιχη διαδικασία θα ακολουθήσουμε και για το σύνολο των κειμένων. Όμως πρώτα θα πρέπει να κρατήσουμε τη θέση , βασικά την αρχή και το τέλος κάθε token. Αυτό είναι απαραίτητο, καθώς στα χαρακτηρισμένα δεδομένα πέρα από τις οντότητες που περιέχονται σε ένα κείμενο πρέπει να υπάχρουν και οι θέσεις εμφάνισης αυτών.

Επίσης, για να επιταχυνθεί η διαδικασία και να αποφευχθεί το ταίριασμα κάποιου όρου αναζήτησης με μία stop word, οι τελευταίες απορρίπτονται πριν αρχίσει η διαδικασία της αναζήτησης. Με αυτόν τον τρόπο γλιτώνουμε κάποιες άσκοπες συγκρίσεις.

In [21]:
stop_words = nlp.Defaults.stop_words
stop_words |= {'.',',',';','?',':','!',' ','&','/','ή','-','(',')','[',']','{','}','"','"','`','~','\xa0'}

In [42]:
import numpy as np

df_texts['entities'] = np.empty((len(df_texts), 0)).tolist()

# for each text
for i in range(df_texts.shape[0]):
    
    text = df_texts.iloc[i]['text']
    text2 = nlp(text)
    tokens_list = []
    entities = []
    
    # keep token, text of token, start and end index of token in text
    for token in text2:
        tokens_list += [(token, token.text, token.idx, token.idx + len(token))]
    
    # remove stop words
    tokens = [(a,b,c,d) for (a,b,c,d) in tokens_list if not(a.is_stop or (a.lower_ in stop_words) or (a.lemma_ in stop_words))]
    
    # preparation and stemming of each token
    final_tokens = [(a, stemmer.stem(b.replace('ά', 'α').replace('έ', 'ε').replace('ή', 'η').replace('ί', 'ι').replace('ό', 'ο')
                                     .replace('ύ', 'υ').replace('ώ', 'ω').replace('ϊ', 'ι').replace('ϋ', 'υ').replace('ΐ', 'ι')
                                     .replace('ΰ', 'υ').replace('Ά', 'Α').replace('Έ', 'Ε').replace('Ή', 'Η').replace('Ί', 'Ι')
                                     .replace('Ό', 'Ο').replace('Ύ', 'Υ').replace('Ϊ', 'Ι').replace('Ϋ', 'Υ').upper()), c, d) 
                    for (a,b,c,d) in tokens]
    
    for (token, token_string, start_pos, end_pos) in final_tokens:
        for term in organs_look_up_list:
            if  token_string == term:
                entities += [(start_pos, end_pos, 'ORGAN')]
    df_texts.at[i,'entities'] = entities

In [45]:
df_texts

,link,text,entities
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...,"[(319, 324, ORGAN), (525, 533, ORGAN), (1156, ..."
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...,"[(247, 252, ORGAN), (851, 858, ORGAN), (1222, ..."
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...,"[(1287, 1294, ORGAN)]"
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...,"[(1351, 1359, ORGAN), (2408, 2416, ORGAN), (24..."
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...,"[(541, 547, ORGAN), (552, 558, ORGAN), (761, 7..."
...,...,...,...
663,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...,"[(2610, 2620, ORGAN), (2832, 2837, ORGAN), (29..."
664,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...,"[(2608, 2618, ORGAN), (2948, 2954, ORGAN)]"
665,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...,"[(863, 871, ORGAN), (1013, 1019, ORGAN), (1050..."
666,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of...","[(1494, 1503, ORGAN), (1594, 1601, ORGAN), (16..."


In [46]:
for i in range(df_texts.shape[0]):
    print(df_texts.iloc[i]['link'],df_texts.iloc[i]['entities'])

https://www.proionta-tis-fisis.com/10-apo-tis-ekpliktikes-hriseis-tis-alois/ [(319, 324, 'ORGAN'), (525, 533, 'ORGAN'), (1156, 1163, 'ORGAN'), (1761, 1767, 'ORGAN'), (1846, 1853, 'ORGAN'), (2040, 2047, 'ORGAN'), (2109, 2115, 'ORGAN'), (2722, 2727, 'ORGAN'), (3236, 3242, 'ORGAN'), (3313, 3319, 'ORGAN'), (3514, 3520, 'ORGAN'), (3590, 3595, 'ORGAN'), (3613, 3619, 'ORGAN'), (3694, 3700, 'ORGAN'), (3889, 3892, 'ORGAN'), (3924, 3930, 'ORGAN')]
https://www.proionta-tis-fisis.com/10-hriseis-gia-tin-iperohi-levanta/ [(247, 252, 'ORGAN'), (851, 858, 'ORGAN'), (1222, 1227, 'ORGAN'), (1244, 1251, 'ORGAN'), (1259, 1266, 'ORGAN'), (1365, 1371, 'ORGAN'), (1386, 1391, 'ORGAN'), (2081, 2086, 'ORGAN'), (2911, 2918, 'ORGAN')]
https://www.proionta-tis-fisis.com/10-logi-gia-na-fitepsis-rigani-sti-glastra/ [(1287, 1294, 'ORGAN')]
https://www.proionta-tis-fisis.com/10-votana-pou-mporoun-na-nikoun-tis-flegmones/ [(1351, 1359, 'ORGAN'), (2408, 2416, 'ORGAN'), (2421, 2427, 'ORGAN'), (2783, 2790, 'ORGAN'), (4989

https://www.proionta-tis-fisis.com/melissohorto-gia-nevriko-systima-stres-aypnies-erpi-ypertasi-kardiotonotiko-iremistiko/ [(28, 35, 'ORGAN'), (127, 134, 'ORGAN'), (257, 264, 'ORGAN'), (1200, 1206, 'ORGAN'), (1524, 1530, 'ORGAN'), (2217, 2225, 'ORGAN'), (3231, 3240, 'ORGAN'), (3930, 3935, 'ORGAN'), (4036, 4041, 'ORGAN'), (4211, 4217, 'ORGAN'), (5221, 5228, 'ORGAN'), (5405, 5411, 'ORGAN'), (5458, 5465, 'ORGAN'), (6546, 6555, 'ORGAN'), (6614, 6623, 'ORGAN')]
https://www.proionta-tis-fisis.com/melissohorto-pafsiponi-kai-antiflegmonodi-drasi-tononei-gastrenteriko-kardiaggeiako-kai-nevriko-systima/ [(100, 107, 'ORGAN'), (211, 218, 'ORGAN'), (353, 360, 'ORGAN'), (1887, 1894, 'ORGAN'), (2067, 2074, 'ORGAN'), (2749, 2756, 'ORGAN'), (3042, 3051, 'ORGAN'), (3125, 3131, 'ORGAN'), (3151, 3161, 'ORGAN'), (3704, 3711, 'ORGAN'), (4116, 4124, 'ORGAN'), (4171, 4179, 'ORGAN'), (4199, 4206, 'ORGAN'), (4499, 4505, 'ORGAN'), (4774, 4780, 'ORGAN'), (5079, 5087, 'ORGAN'), (6151, 6159, 'ORGAN'), (6830, 6839, 

https://www.proionta-tis-fisis.com/thymari-pafsipones-kai-antiflegmonodeis-idiotites-horis-parenergeies/ [(2862, 2872, 'ORGAN')]
https://www.proionta-tis-fisis.com/thymari-thymelaio-kai-i-drasi-tous-enantia-ston-karkino/ [(506, 513, 'ORGAN'), (519, 528, 'ORGAN'), (537, 543, 'ORGAN'), (708, 715, 'ORGAN'), (2015, 2023, 'ORGAN'), (2046, 2053, 'ORGAN'), (2046, 2053, 'ORGAN'), (2253, 2261, 'ORGAN'), (2298, 2306, 'ORGAN'), (3105, 3111, 'ORGAN'), (3231, 3237, 'ORGAN'), (3640, 3646, 'ORGAN'), (3839, 3845, 'ORGAN'), (3904, 3910, 'ORGAN'), (3931, 3938, 'ORGAN'), (4068, 4075, 'ORGAN'), (4334, 4341, 'ORGAN'), (4515, 4522, 'ORGAN')]
https://www.proionta-tis-fisis.com/thymari-to-votano-me-tin-ishyroteri-antimikrobiaki-kai-antisiptiki-drasi/ [(2513, 2520, 'ORGAN'), (3311, 3317, 'ORGAN'), (3852, 3860, 'ORGAN'), (4189, 4194, 'ORGAN'), (4189, 4194, 'ORGAN'), (5455, 5463, 'ORGAN'), (5492, 5500, 'ORGAN'), (5811, 5820, 'ORGAN'), (6652, 6661, 'ORGAN')]
https://www.proionta-tis-fisis.com/ti-einai-agleouras-m

In [47]:
df_texts.iloc[0]['entities']

[(319, 324, 'ORGAN'),
 (525, 533, 'ORGAN'),
 (1156, 1163, 'ORGAN'),
 (1761, 1767, 'ORGAN'),
 (1846, 1853, 'ORGAN'),
 (2040, 2047, 'ORGAN'),
 (2109, 2115, 'ORGAN'),
 (2722, 2727, 'ORGAN'),
 (3236, 3242, 'ORGAN'),
 (3313, 3319, 'ORGAN'),
 (3514, 3520, 'ORGAN'),
 (3590, 3595, 'ORGAN'),
 (3613, 3619, 'ORGAN'),
 (3694, 3700, 'ORGAN'),
 (3889, 3892, 'ORGAN'),
 (3924, 3930, 'ORGAN')]

In [44]:
entities

[(1298, 1305, 'ORGAN'),
 (1470, 1474, 'ORGAN'),
 (2913, 2921, 'ORGAN'),
 (2913, 2921, 'ORGAN'),
 (3011, 3019, 'ORGAN'),
 (3011, 3019, 'ORGAN'),
 (3387, 3397, 'ORGAN'),
 (3387, 3397, 'ORGAN'),
 (3913, 3923, 'ORGAN'),
 (4502, 4508, 'ORGAN')]

Στη συνέχεια, ακολουθούμε την ίδια διαδικασία για την επισήμανση των βοτάνων στα διαθέσιμα κείμενα.

Πρώτα, εξάγουμε μια λίστα γνωστών βοτάνων. Πηγές:[alfavita](https://www.alfavita.gr/epistimi/179616_pliris-katalogos-botanon-kai-oi-therapeies-toys-mathe-tis-astheneies-poy-therapeyei) και όλα τα λεξικά από Α έως Ω στο [proionta-tis-fisis](https://www.proionta-tis-fisis.com/to-lexiko-ton-votanon-ola-apo-a/).

In [49]:
with open('../data/herbs.txt','r',encoding='utf-8') as f:
    content = f.readlines()
content = [x.strip() for x in content] 

In [50]:
for herbs_string in content:
    herbs = herbs_string.split(',')

In [51]:
herbs

['αβρωνία',
 'αγκινάρα',
 'αγιόκλημα',
 'αλθαία',
 'αγγελική',
 'αγγουριά',
 'αγιάγκαθο',
 'κνίκος',
 'αγριάδα',
 'αγριμόνιο',
 'αλόη',
 'αγριοκαστανιά',
 'αδίαντο',
 'άδωνις',
 'άζα',
 'αζαρο',
 'άκανθος',
 'ακόνιτο',
 'άκορος',
 'αλχεμίλη',
 'αμαμελίδα',
 'αμάραντος',
 'άμπελος',
 'αμυγδαλιά',
 'άνηθος',
 'απήγαγος',
 'άρκευθος',
 'άρνικα',
 'ασπερούλα',
 'αστράγαλος',
 'αχιλλέα',
 'αχλαδιά',
 'αψιθιά',
 'αρτεμισία',
 'βάλσαμο',
 'βασιλικός',
 'βατομουριά',
 'βελανιδιά',
 'βέρατρο',
 'βερβένα',
 'βερβερίδα',
 'βερόνικα',
 'βιβούρνο',
 'βρώμη',
 'βρυωνιά',
 'βρωμελαϊνη',
 'βαλεριάνα',
 'σπαθόχορτο',
 'γαρίφαλο',
 'γαρύφαλλο',
 'γέο',
 'γεράνι',
 'γκαλέγκα',
 'γκι',
 'ιξός',
 'γλομπουλάρια',
 'γλιστρίδα',
 'γλυκάνισος',
 'γλυκόριζα',
 'γανόδερμα',
 'γαϊδουράγκαθο',
 'δατούρα',
 'διαβολόχορτο',
 'δάφνη',
 'δαφνοκερασιά',
 'δαχτυλίδα',
 'δελφίνιο',
 'δενδρολίβανο',
 'δίκταμο',
 'δίψακος',
 'δρακοντιά',
 'δυόσμος',
 'εκουιζέτο',
 'πολυκόμπι',
 'ερίγκερο',
 'ερυθραία',
 'ευκάλυπτος',
 'ευώ

In [54]:
herbs_look_up_list = []
for search_term in herbs:
    search_term = search_term.replace('ά', 'α')
    search_term = search_term.replace('έ', 'ε')
    search_term = search_term.replace('ή', 'η')
    search_term = search_term.replace('ί', 'ι')
    search_term = search_term.replace('ό', 'ο')
    search_term = search_term.replace('ύ', 'υ')
    search_term = search_term.replace('ώ', 'ω')
    search_term = search_term.replace('ϊ', 'ι')
    search_term = search_term.replace('ΐ', 'ι')
    search_term = search_term.replace('ϋ', 'υ')
    search_term = search_term.replace('ΰ', 'υ')
    search_term = search_term.upper()
    herbs_look_up_list += [stemmer.stem(search_term)]

In [55]:
herbs_look_up_list

['ΑΒΡΩΝ',
 'ΑΓΚΙΝΑΡ',
 'ΑΓΙΟΚΛΗΜ',
 'ΑΛΘΑΙ',
 'ΑΓΓΕΛ',
 'ΑΓΓΟΥΡ',
 'ΑΓΙΑΓΚΑΘ',
 'ΚΝ',
 'ΑΓΡΙΑΔ',
 'ΑΓΡΙΜΟΝ',
 'ΑΛΟ',
 'ΑΓΡΙΟΚΑΣΤΑΝ',
 'ΑΔΙΑΝΤ',
 'ΑΔΩΝΙΣ',
 'ΑΖ',
 'ΑΖΑΡ',
 'ΑΚΑΝΘ',
 'ΑΚΟΝΙΤ',
 'ΑΚΟΡ',
 'ΑΛΧΕΜΙΛ',
 'ΑΜΑΜΕΛΙΔ',
 'ΑΜΑΡΑΝΤ',
 'ΑΜΠΕΛ',
 'ΑΜΥΓΔΑΛ',
 'ΑΝΗΘ',
 'ΑΠΗΓΑΓ',
 'ΑΡΚΕΥΘ',
 'ΑΡΝ',
 'ΑΣΠΕΡΟΥΛ',
 'ΑΣΤΡΑΓΑΛ',
 'ΑΧΙΛΛ',
 'ΑΧΛΑΔ',
 'ΑΨΙΘ',
 'ΑΡΤΕΜΙΣ',
 'ΒΑΛΣΑΜ',
 'ΒΑΣΙΛ',
 'ΒΑΤΟΜΟΥΡ',
 'ΒΕΛΑΝΙΔ',
 'ΒΕΡΑΤΡ',
 'ΒΕΡΒΕΝ',
 'ΒΕΡΒΕΡΙΔ',
 'ΒΕΡΟΝ',
 'ΒΙΒΟΥΡΝ',
 'ΒΡΩΜ',
 'ΒΡΥΩΝ',
 'ΒΡΩΜΕΛΑΙΝ',
 'ΒΑΛΕΡΙΑΝ',
 'ΣΠΑΘΟΧΟΡΤ',
 'ΓΑΡΙΦΑΛ',
 'ΓΑΡΥΦΑΛΛ',
 'ΓΕ',
 'ΓΕΡΑΝ',
 'ΓΚΑΛΕΓΚ',
 'ΓΚ',
 'ΙΞ',
 'ΓΛΟΜΠΟΥΛΑΡ',
 'ΓΛΙΣΤΡΙΔ',
 'ΓΛΥΚΑΝΙΣ',
 'ΓΛΥΚΟΡΙΖ',
 'ΓΑΝΟΔΕΡΜ',
 'ΓΑΙΔΟΥΡΑΓΚΑΘ',
 'ΔΑΤΟΥΡ',
 'ΔΙΑΒΟΛΟΧΟΡΤ',
 'ΔΑΦΝ',
 'ΔΑΦΝΟΚΕΡΑΣ',
 'ΔΑΧΤΥΛΙΔ',
 'ΔΕΛΦΙΝ',
 'ΔΕΝΔΡΟΛΙΒΑΝ',
 'ΔΙΚΤΑΜ',
 'ΔΙΨΑΚ',
 'ΔΡΑΚΟΝΤ',
 'ΔΥΟΣΜ',
 'ΕΚΟΥΙΖΕΤ',
 'ΠΟΛΥΚΟΜΠ',
 'ΕΡΙΓΚΕΡ',
 'ΕΡΥΘΡΑΙ',
 'ΕΥΚΑΛΥΠΤ',
 'ΕΥΩΝΥΜ',
 'ΕΦΕΔΡ',
 'ΕΧΙΝΑΚΕΙ',
 'ΕΛΑΙΟΛΑΔ',
 'ΖΑΜΠΟΥΚ',
 'ΣΑΜΠΟΥΚ',
 'ΗΛΙΑΝΘ',
 'ΗΛΙΟΤΡΟΠ',
 'ΘΡΟΥΜΠ',
 'ΘΑΨ',
 'ΘΥΜΑΡ'

Στη συνέχεια, ακολουθούμε την ίδια διαδικασία για τα κείμενα με την περίπτωση των οργάνων.

In [59]:
# for each text
for i in range(df_texts.shape[0]):
    
    text = df_texts.iloc[i]['text']
    text2 = nlp(text)
    tokens_list = []
    entities = []
    
    # keep token, text of token, start and end index of token in text
    for token in text2:
        tokens_list += [(token, token.text, token.idx, token.idx + len(token))]
    
    # remove stop words
    tokens = [(a,b,c,d) for (a,b,c,d) in tokens_list if not(a.is_stop or (a.lower_ in stop_words) or (a.lemma_ in stop_words))]
    
    # preparation and stemming of each token
    final_tokens = [(a, stemmer.stem(b.replace('ά', 'α').replace('έ', 'ε').replace('ή', 'η').replace('ί', 'ι').replace('ό', 'ο')
                                     .replace('ύ', 'υ').replace('ώ', 'ω').replace('ϊ', 'ι').replace('ϋ', 'υ').replace('ΐ', 'ι')
                                     .replace('ΰ', 'υ').replace('Ά', 'Α').replace('Έ', 'Ε').replace('Ή', 'Η').replace('Ί', 'Ι')
                                     .replace('Ό', 'Ο').replace('Ύ', 'Υ').replace('Ϊ', 'Ι').replace('Ϋ', 'Υ').upper()), c, d) 
                    for (a,b,c,d) in tokens]
    
    for (token, token_string, start_pos, end_pos) in final_tokens:
        for term in herbs_look_up_list:
            if  token_string == term:
                entities += [(start_pos, end_pos, 'HERB')]
    df_texts.at[i,'entities'] = df_texts.iloc[i]['entities'] + entities

In [62]:
df_texts

,link,text,entities
0,https://www.proionta-tis-fisis.com/10-apo-tis-...,Βότανα 10 από τις εκπληκτικές χρήσεις της α...,"[(319, 324, ORGAN), (525, 533, ORGAN), (1156, ..."
1,https://www.proionta-tis-fisis.com/10-hriseis-...,Βότανα 10 χρήσεις για την υπέροχη λεβάνταΒό...,"[(247, 252, ORGAN), (851, 858, ORGAN), (1222, ..."
2,https://www.proionta-tis-fisis.com/10-logi-gia...,Βότανα 10 λόγοι για να φυτέψεις ρίγανη στη ...,"[(1287, 1294, ORGAN), (35, 41, HERB), (93, 99,..."
3,https://www.proionta-tis-fisis.com/10-votana-p...,Βότανα 10 βότανα που μπορούν να νικούν τις ...,"[(1351, 1359, ORGAN), (2408, 2416, ORGAN), (24..."
4,https://www.proionta-tis-fisis.com/15-logoi-na...,Βότανα 15 λόγοι να βάλεις στην ζωή σου το Γ...,"[(541, 547, ORGAN), (552, 558, ORGAN), (761, 7..."
...,...,...,...
663,https://www.proionta-tis-fisis.com/vrionia-i-k...,Βότανα Βρυωνία η κρητική - ΙδιότητεςΒόταναΒ...,"[(2610, 2620, ORGAN), (2832, 2837, ORGAN), (29..."
664,https://www.proionta-tis-fisis.com/vrouves-sin...,Βότανα Βρούβες-Σινάπι και διατροφική αξία~Τ...,"[(2608, 2618, ORGAN), (2948, 2954, ORGAN), (24..."
665,https://www.proionta-tis-fisis.com/ydrastis-gi...,Βότανα Υδραστίς για λοιμώξεις του αναπνευστ...,"[(863, 871, ORGAN), (1013, 1019, ORGAN), (1050..."
666,https://www.proionta-tis-fisis.com/yssopos-o-f...,"Βότανα Ύσσωπος ο φαρμακευτικός, Hyssopus of...","[(1494, 1503, ORGAN), (1594, 1601, ORGAN), (16..."


In [63]:
for i in range(df_texts.shape[0]):
    print(df_texts.iloc[i]['link'],df_texts.iloc[i]['entities'])

https://www.proionta-tis-fisis.com/10-apo-tis-ekpliktikes-hriseis-tis-alois/ [(319, 324, 'ORGAN'), (525, 533, 'ORGAN'), (1156, 1163, 'ORGAN'), (1761, 1767, 'ORGAN'), (1846, 1853, 'ORGAN'), (2040, 2047, 'ORGAN'), (2109, 2115, 'ORGAN'), (2722, 2727, 'ORGAN'), (3236, 3242, 'ORGAN'), (3313, 3319, 'ORGAN'), (3514, 3520, 'ORGAN'), (3590, 3595, 'ORGAN'), (3613, 3619, 'ORGAN'), (3694, 3700, 'ORGAN'), (3889, 3892, 'ORGAN'), (3924, 3930, 'ORGAN'), (187, 191, 'HERB'), (813, 817, 'HERB'), (966, 971, 'HERB'), (1035, 1039, 'HERB'), (1409, 1414, 'HERB'), (1870, 1874, 'HERB'), (1881, 1890, 'HERB'), (1906, 1910, 'HERB'), (2211, 2215, 'HERB'), (2364, 2368, 'HERB'), (2572, 2576, 'HERB'), (2666, 2671, 'HERB'), (2847, 2851, 'HERB'), (3211, 3215, 'HERB'), (3548, 3552, 'HERB'), (3831, 3835, 'HERB'), (4125, 4129, 'HERB'), (4239, 4245, 'HERB'), (4394, 4399, 'HERB'), (4460, 4464, 'HERB'), (4478, 4484, 'HERB'), (4854, 4859, 'HERB'), (4891, 4895, 'HERB'), (4917, 4922, 'HERB'), (4981, 4985, 'HERB'), (5045, 5049, '

https://www.proionta-tis-fisis.com/ekatofyllo-i-magiatiki-triantafyllia-kai-oi-idiotites-tis/ [(787, 791, 'ORGAN'), (825, 830, 'ORGAN'), (839, 845, 'ORGAN'), (1673, 1678, 'ORGAN'), (2268, 2274, 'ORGAN'), (2605, 2611, 'ORGAN'), (3993, 3998, 'ORGAN'), (4134, 4139, 'ORGAN'), (4179, 4186, 'ORGAN'), (10, 20, 'HERB'), (10, 20, 'HERB'), (34, 47, 'HERB'), (98, 111, 'HERB'), (170, 180, 'HERB'), (170, 180, 'HERB'), (194, 207, 'HERB'), (272, 281, 'HERB'), (576, 590, 'HERB'), (755, 767, 'HERB'), (1013, 1027, 'HERB'), (1163, 1171, 'HERB'), (1183, 1202, 'HERB'), (1310, 1322, 'HERB'), (1393, 1405, 'HERB'), (1623, 1635, 'HERB'), (1687, 1695, 'HERB'), (1705, 1724, 'HERB'), (1762, 1775, 'HERB'), (2916, 2924, 'HERB'), (2959, 2971, 'HERB'), (3130, 3144, 'HERB'), (3881, 3895, 'HERB'), (4326, 4340, 'HERB'), (4725, 4729, 'HERB'), (4731, 4741, 'HERB'), (4743, 4747, 'HERB'), (4781, 4788, 'HERB'), (5239, 5253, 'HERB')]
https://www.proionta-tis-fisis.com/ekhylisma-fyllon-elias-apotrepei-kai-therapevei-plithos-pa

https://www.proionta-tis-fisis.com/pos-i-kannavi-therapevi-ton-karkino-run-from-the-cure-vinteo/ [(1063, 1069, 'ORGAN'), (1617, 1623, 'ORGAN'), (2084, 2093, 'ORGAN'), (2177, 2183, 'ORGAN'), (16, 23, 'HERB'), (97, 104, 'HERB'), (187, 194, 'HERB'), (423, 431, 'HERB'), (1018, 1026, 'HERB'), (1376, 1383, 'HERB'), (1571, 1579, 'HERB'), (1775, 1783, 'HERB'), (2459, 2466, 'HERB'), (2562, 2570, 'HERB'), (2571, 2578, 'HERB')]
https://www.proionta-tis-fisis.com/pos-na-ehete-kathe-mera-freska-votana-stin-kouzina-sas/ [(2134, 2140, 'HERB')]
https://www.proionta-tis-fisis.com/pos-na-exisorropisete-tis-ormones-sas-me-fysiko-tropo/ [(1538, 1546, 'ORGAN'), (2838, 2848, 'ORGAN'), (3296, 3308, 'ORGAN'), (3397, 3406, 'ORGAN'), (3411, 3422, 'ORGAN'), (3453, 3460, 'ORGAN'), (3469, 3476, 'ORGAN'), (3817, 3829, 'ORGAN'), (3940, 3945, 'ORGAN'), (6774, 6781, 'ORGAN'), (6787, 6792, 'ORGAN'), (6845, 6852, 'ORGAN'), (8228, 8240, 'ORGAN'), (9356, 9365, 'ORGAN'), (9412, 9417, 'ORGAN'), (2447, 2451, 'HERB'), (2943, 

https://www.proionta-tis-fisis.com/vasilikos-droseros-ke-piperatos/ [(2621, 2627, 'ORGAN'), (2928, 2936, 'ORGAN'), (3236, 3243, 'ORGAN'), (3655, 3658, 'ORGAN'), (3810, 3815, 'ORGAN'), (10, 19, 'HERB'), (119, 128, 'HERB'), (437, 446, 'HERB'), (566, 575, 'HERB'), (673, 682, 'HERB'), (735, 744, 'HERB'), (811, 820, 'HERB'), (861, 867, 'HERB'), (869, 875, 'HERB'), (877, 884, 'HERB'), (960, 966, 'HERB'), (990, 996, 'HERB'), (1018, 1025, 'HERB'), (1027, 1035, 'HERB'), (1077, 1083, 'HERB'), (1130, 1136, 'HERB'), (1141, 1145, 'HERB'), (1209, 1215, 'HERB'), (1847, 1856, 'HERB'), (2010, 2019, 'HERB'), (2282, 2291, 'HERB'), (2398, 2407, 'HERB'), (2471, 2480, 'HERB'), (2566, 2570, 'HERB'), (2575, 2583, 'HERB'), (2642, 2651, 'HERB'), (2885, 2894, 'HERB'), (2949, 2958, 'HERB'), (3059, 3068, 'HERB'), (3363, 3367, 'HERB'), (3560, 3564, 'HERB'), (3886, 3894, 'HERB'), (4116, 4122, 'HERB'), (4144, 4153, 'HERB'), (4224, 4230, 'HERB'), (4311, 4317, 'HERB'), (4325, 4333, 'HERB'), (4460, 4466, 'HERB'), (4617,

In [64]:
df_texts.to_csv('../data/proionta_tis_fisis_votana_entities.csv', index=False)